In [11]:
import pandas as pd
import numpy as np

In [12]:
train = pd.read_csv('train.csv',index_col=None,header=None)
# test = pd.read_csv('test.csv',index_col=None,header=None)

In [13]:
Y_trainmin = train.drop([0,1,2,4],axis=1)
Y_trainmax = train.drop([0,1,2,3],axis=1)
X_train = train.drop([3,4],axis=1)

# Y_testmin = test.drop([0,1,2,4],axis=1)
# Y_testmax = test.drop([0,1,2,3],axis=1)
# X_test = test.drop([3,4],axis=1)

In [14]:
# print("Number transactions train dataset: ", len(X_train))
# print("Number transactions test dataset: ", len(X_test))
# print("Total number of transactions: ", len(X_train)+len(X_test))

In [15]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import sklearn.metrics
from sklearn.datasets import make_regression
import seaborn as sns

In [16]:
# print(Y_trainmax.max(axis=0))
# print(Y_trainmin.max(axis=0))
# print(Y_testmax.max(axis=0))
# print(Y_testmin.max(axis=0))

### GBDT 

In [17]:
regr = GradientBoostingRegressor(max_depth=20, random_state=0,max_features=2,n_estimators=333)
regr.fit(X_train, Y_trainmin/90000)

regr2 = GradientBoostingRegressor(max_depth=20, random_state=0,max_features=2,n_estimators=333)
regr2.fit(X_train, Y_trainmax/100000)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
y_pred = regr.predict(X_train.values)
print('最小薪酬训练集均方根误差',np.sqrt(mean_squared_error(Y_trainmin.values.T[0], y_pred*90000)))
# y_pred = regr.predict(X_test.values)
# print('最小薪酬测试集均方根误差',np.sqrt(mean_squared_error(Y_testmin.values.T[0], y_pred*90000)))
# print('最小薪酬R square',r2_score(Y_testmin.values.T[0], y_pred*90000))#衡量正确率

y_pred2 = regr2.predict(X_train.values)
print('最大薪酬训练集均方根误差',np.sqrt(mean_squared_error(Y_trainmax.values.T[0], y_pred2*100000)))
# y_pred2 = regr.predict(X_test.values)
# print('最大薪酬测试集均方根误差',np.sqrt(mean_squared_error(Y_testmax.values.T[0], y_pred2*100000)))
# print('最大薪酬R square',r2_score(Y_testmax.values.T[0], y_pred2*100000))#衡量正确率


D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


最小薪酬训练集均方根误差 3211.86913268
最大薪酬训练集均方根误差 5501.38145387


In [18]:
minsalary = y_pred*90000
maxsalary= y_pred2*100000

In [19]:
Y_trainmax.columns =['最高工资']
Y_trainmin.columns =['最低工资']
Y_trainmax.to_csv('trainmaxsalary.csv',index=None)
Y_trainmin.to_csv('trainminsalary.csv',index=None)

In [20]:
#预测值
file = open('pretrainmaxsalary.csv','a')
for i in range(len(maxsalary)):
    s = str(maxsalary[i]).replace('[','').replace(']','')#去除[],这两行按数据不同，可以选择
    s = s.replace("'",'').replace(',','') +'\n'   #去除单引号，逗号，每行末尾追加换行符
    file.write(s)
file.close()
file = open('pretrainminsalary.csv','a')
for i in range(len(minsalary)):
    s = str(minsalary[i]).replace('[','').replace(']','')#去除[],这两行按数据不同，可以选择
    s = s.replace("'",'').replace(',','') +'\n'   #去除单引号，逗号，每行末尾追加换行符
    file.write(s)
file.close()



In [21]:
maxsalary = pd.read_csv('pretrainmaxsalary.csv',header=None)
minsalary = pd.read_csv('pretrainminsalary.csv',header=None)

In [22]:
# minsalary.columns =['预测最高工资']
# minsalary.columns =['预测最低工资']

In [23]:
X_train.columns =['职位','城市','经验']

In [24]:
X_train['预测最低工资']=minsalary
X_train['预测最高工资']=maxsalary
X_train.to_csv('result/salaryresult.csv',index=None,encoding="utf_8_sig")

In [25]:
#将职位城市经验从字典中转换出来
city = pd.read_pickle('dict/city.pkl')
experience = pd.read_pickle('dict/experience.pkl')
occupation = pd.read_pickle('dict/occupation.pkl')
salary = pd.read_csv('result/salaryresult.csv')

precity =[]
for i in salary['城市']:
    for key,values in city.items():
        if values == i:
            precity.append(key)
salary['城市'] = precity

preexperience =[]
for i in salary['经验']:
    for key,values in experience.items():
        if values == i:
            preexperience.append(key)
#print(preexperience)
salary['经验'] = preexperience

preoccupation =[]
for i in salary['职位']:
    for key,values in occupation.items():
        if values == i:
            preoccupation.append(key)
#print(preexperience)
salary['职位'] = preoccupation

salary.to_csv('result/Resultsalary.csv',index=None,encoding="utf_8_sig")